In [1]:
# to make the export go to the fitbit app, click on your icon, and order a google takeout 
# took like 8 hours the last time

In [11]:
import os
# Get the list of all files and directories
workingDataPath = "/home/chowder/Documents/workingData/fitbit/hr/"

pathOfExport = "/home/chowder/Documents/dataExports/fitbit/"
exportFilePath = "27-8-24/takeout-20240828T033834Z-001/Takeout/Fitbit/Global Export Data"

dir_list = os.listdir(pathOfExport + exportFilePath)

In [3]:
hrFilenames = [x for x in dir_list if x.split("_")[0] == "heart"]
hrFilenames[0]

'heart_rate-2021-11-22.json'

In [4]:
#takes like 40 minutes to make 15.8 million rows
import json
import pandas as pd

columnNames = ["sampleDT", "confidence", "value"]

samplesCount = 0
samplesList = []

for fn in hrFilenames:
    data = json.load(open(pathOfExport + exportFilePath + "/" + fn))
    for row in data:
        samplesList.append([pd.to_datetime(row["dateTime"] + "-0700"), row["value"]["confidence"], row["value"]["bpm"]])
        samplesCount += 1
        if samplesCount % 300_000 == 0:
            print(samplesCount)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000
1

In [27]:
samplesList = sorted(samplesList, key=lambda x: x[0]) #sort by timestamp

In [28]:
import pandas as pd

fitbitHRdf = pd.DataFrame(data=samplesList, columns=columnNames)
fitbitHRdf = fitbitHRdf.set_index("sampleDT")
fitbitHRdf["confidence"] = fitbitHRdf["confidence"].astype('uint8')
fitbitHRdf["value"] = fitbitHRdf["value"].astype('uint8')
fitbitHRdf.dtypes

In [37]:
# Get the size of the file in bytes
workingDataCompleteFile = "/home/chowder/Documents/workingData/fitbit/hr/fitbitHRdf.parquet.gzip"
fitbitHRdf.to_parquet(workingDataCompleteFile,
              compression='gzip') 

file_size = os.path.getsize(workingDataCompleteFile)
os.remove(workingDataCompleteFile)

In [36]:
import math
numFiles = math.ceil(file_size / (1024 * 1024 * 5))
rows_per_file = int(len(fitbitHRdf)/numFiles)

print(f"the file size of all the data is about {file_size // (1024 * 1024)} MB")
print(f"the total number of rows in the file is {len(fitbitHRdf)}")
print(f"splitting into {numFiles} number of files with {rows_per_file} rows per file")
print()

for fileNumber in range(numFiles + 1):
    startRow = fileNumber * rows_per_file
    if fileNumber == numFiles:
        endRow = len(fitbitHRdf) - 1
    else:
        endRow = ((fileNumber + 1) * rows_per_file) - 1

    print(f"saving rows {startRow} to {endRow}")
    
    parquetName = str(fitbitHRdf.iloc[startRow].name) +\
                  "_" +\
                  str(fitbitHRdf.iloc[endRow].name) +\
                  ".parquet.gzip"
    print(f"to a file named {parquetName}")

    fitbitHRdf.iloc[startRow:endRow].to_parquet(workingDataPath + parquetName,
              compression='gzip') 


the file size of all the data is about 86 MB
the total number of rows in the file is 15826367
splitting into 18 number of files with 879242 rows per file

saving rows 0 to 879241
to a file named 2020-05-13 17:02:19-07:00_2020-08-02 07:28:31-07:00.parquet.gzip
saving rows 879242 to 1758483
to a file named 2020-08-02 07:28:36-07:00_2020-10-22 19:11:46-07:00.parquet.gzip
saving rows 1758484 to 2637725
to a file named 2020-10-22 19:12:01-07:00_2021-01-11 15:28:28-07:00.parquet.gzip
saving rows 2637726 to 3516967
to a file named 2021-01-11 15:28:33-07:00_2021-03-31 17:03:08-07:00.parquet.gzip
saving rows 3516968 to 4396209
to a file named 2021-03-31 17:03:13-07:00_2021-06-18 16:07:01-07:00.parquet.gzip
saving rows 4396210 to 5275451
to a file named 2021-06-18 16:07:07-07:00_2021-09-06 21:51:24-07:00.parquet.gzip
saving rows 5275452 to 6154693
to a file named 2021-09-06 21:51:34-07:00_2021-11-30 20:39:51-07:00.parquet.gzip
saving rows 6154694 to 7033935
to a file named 2021-11-30 20:39:56-07